In [1]:
google_json_key = 'buying-house-283614-38d7fde831c8.json'

In [2]:
import gspread 
from gspread_dataframe import get_as_dataframe, set_with_dataframe 
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe 

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_json_key, scope) 
gc = gspread.authorize(credentials)

# ===============================

In [9]:
filename = "buy house notes"
worksheet_name = "houses_enschede"

In [10]:
worksheet = gc.open(filename).worksheet(worksheet_name)
df_enschede = get_as_dataframe(worksheet,index_column=0)

In [11]:
df_enschede['makelaars'].value_counts().head(10)

EXTRA Makelaars                   75
Thoma Post Makelaars Enschede     40
Snelder Zijlstra Makelaars        39
Nieuw Wonen Twente                31
Euverman & Nuyts Enschede         25
Prisma Makelaars Enschede B.V.    17
Vera Welker Makelaars             16
Keuter Makelaars                  15
ten Hag Makelaarsgroep            15
Marcel Kon Makelaardij            13
Name: makelaars, dtype: int64

# ===============================

In [6]:
filename = "buy house notes"
worksheet_name = "houses_hengelo"

In [7]:
worksheet = gc.open(filename).worksheet(worksheet_name)
df_hengelo = get_as_dataframe(worksheet,index_column=0)

In [8]:
df_hengelo['makelaars'].value_counts().head(10)

Nieuw Wonen Twente                               32
Zengerink Makelaardij & Hypotheekspecialisten    26
Snelder Zijlstra Makelaars Hengelo               24
Dekkers Makelaars                                23
DES HUIZES MAKELAARS                             22
Prisma Era Makelaars Hengelo                     19
Blik Makelaars B.V.                              16
Thoma Post Makelaars Hengelo                     11
ten Hag makelaarsgroep                            9
Marcel Kon Makelaardij                            5
Name: makelaars, dtype: int64

# ===============================


- Thoma Post Makelaars
- Nieuw Wonen Twente
- Snelder Zijlstra Makelaars
- Prisma
- ten Hag
- Marcel Kon Makelaardij 

# ===============================

# Thoma Post Makelaars

In [99]:
google_json_key = 'buying-house-283614-38d7fde831c8.json'
filename = "buy house notes"
worksheet_name = "houses"
worksheet_name = "houses_thomapost"

In [100]:
import gspread 
from gspread_dataframe import get_as_dataframe, set_with_dataframe 
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe 

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_json_key, scope) 
gc = gspread.authorize(credentials)

worksheet = gc.open(filename).worksheet(worksheet_name)
df_current = get_as_dataframe(worksheet,index_column=0)

# --

In [14]:
city = 'enschede'
url = 'https://thomapost.nl/?s='+city

In [95]:
import pandas as pd 

import cfscrape
from lxml import etree
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
              'Accept-Encoding': 'gzip, deflate, sdch',
              'Accept-Language' : 'en-US,en;q=0.9,nl;q=0.8',
              'Cache-Control' : 'max-age=0',
              'Connection': 'keep-alive',
              'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
scraper = cfscrape.create_scraper()

def thomapost_crawler (city):
    
    url = 'https://thomapost.nl/?s='+city
    scraped_html=scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    df_links = pd.DataFrame(html.xpath("//a/@href"))
    df_links_unique =df_links[df_links[0].str.contains("/woning/"+str(city)+"-")].drop_duplicates()
#   -------------------------------------------
#   -------------------------------------------
    links_unique = list(set(df_links_unique[0]))
    df = pd.DataFrame()
    for url in links_unique:
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        columns = html.xpath("//div[@class='all-details']//dt/text()")
        values = html.xpath("//div[@class='all-details']//dd/text()")
        df_temp = pd.DataFrame([values], columns=[x.lower() for x in columns])
        
        address = html.xpath("//div[@class='content']/p/text()")[0].replace('Enschede ','').replace('â\x80\x93 ','').replace(', ','')
        df_temp['address']= [address]
        df_temp['city'] = [city]
        df_temp['url'] = [url]
        
        df = pd.concat([df,df_temp], axis=0, ignore_index=True)
    return df

In [96]:
df_thomapost = thomapost_crawler('enschede')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [101]:
%run gmaps_bike_distance_duration.ipynb

In [102]:
point_b = 'Enschede Railway Station, Enschede'
# point_b = 'Stationsplein, Hengelo'
 
df_thomapost['bike_time'] = df_thomapost['address'].apply(lambda x: gmaps_bike_time(gmaps_key,x+', enschede',point_b))

Madoerastraat 3, enschede -> Enschede Railway Station, Enschede : 6 mins
Schildkampstraat 19, enschede -> Enschede Railway Station, Enschede : 21 mins
Van der Waalslaan 1, enschede -> Enschede Railway Station, Enschede : 10 mins
Oldenzaalsestraat 2-6, enschede -> Enschede Railway Station, Enschede : 4 mins
Stentorlaan 26, enschede -> Enschede Railway Station, Enschede : 25 mins
Terschellinglaan 13, enschede -> Enschede Railway Station, Enschede : 12 mins
Zwarteweg 8, enschede -> Enschede Railway Station, Enschede : 20 mins
Burgemeester Edo Bergsmalaan 44, enschede -> Enschede Railway Station, Enschede : 4 mins
Dennenweg 148, enschede -> Enschede Railway Station, Enschede : 9 mins
Beckumerstraat 46-46a, enschede -> Enschede Railway Station, Enschede : 27 mins
Boulevard 1945 230, enschede -> Enschede Railway Station, Enschede : 5 mins


In [105]:
set_with_dataframe(worksheet, df_thomapost[['url','address','bike_time', 'city','koopprijs', 'status', 'aanvaarding', 'huidig gebruik', 'bouwvorm',
       'bouwjaar', 'woonoppervlakte', 'perceel oppervlakte', 'inhoud',
       'aantal kamers', 'aantal slaapkamers', 'voorzieningen']])

# =================================================

# Snelder Zijlstra
not possible

In [107]:
max_price = 550000
city = 'enschede'

url = 'https://snelderzijlstra.nl/wonen/aanbod?buyOrRent=buy&maxPrice='+str(max_price)+'&houseType=woonhuis&gardenType=achtertuin&office='+city+'&textSearch='+city+'&orderBy=created_at&orderDirection=desc'

In [108]:
scraped_html=scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)
df_links = pd.DataFrame(html.xpath("//a/@href"))

In [113]:
print(scraped_html.decode("utf-8"))

<!DOCTYPE html>
<html lang="nl">
	
<head>
    <meta charset="utf-8"/>    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=0">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <title>Woningaanbod in Twente - Snelder Zijlstra Makelaars</title><link href="https://snelderzijlstra.nl/wonen/aanbod" rel="canonical"/><meta name="description" content="Bekijk hier het actuele woningaanbod van Snelder Zijlstra Makelaars! Plan vrijblijvend een bezichtiging of plaats een zoekopdracht.
"/><meta property="og:title" content="Woningaanbod in Twente"/><meta property="twitter:title" content="Woningaanbod in Twente"/><meta property="og:description" content="Bekijk hier het actuele woningaanbod van Snelder Zijlstra Makelaars! Plan vrijblijvend een bezichtiging of plaats een zoekopdracht.
"/><meta property="twitter:description" content="Bekijk hier het actuele woningaanbod van Snelder Zijlstra Makelaars! Plan vrijblijvend een bezichtiging of pla